In [1]:
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,  mean_squared_error, f1_score, roc_auc_score


##### Data loading and Preparation

In [2]:
train_df = pd.read_csv("Train.csv")

In [3]:
train_df.shape

(7736, 21)

In [4]:
train_df.head()

ID  gender  age  ... job_or_business  nature_of_work target
0  ID_00008683  Female   20  ...              No  Not applicable      0
1  ID_00061163  Female   42  ...  Not applicable       Permanent      0
2  ID_00071403  Female   86  ...              No  Not applicable      0
3  ID_00077803  Female   20  ...              No  Not applicable      0
4  ID_00086763  Female   26  ...              No  Not applicable      0

[5 rows x 21 columns]

In [5]:
train_df.columns

Index(['ID', 'gender', 'age', 'race', 'dwelling', 'dwelling_type',
       'province_code', 'metro_code', 'psu', 'nationality', 'RTH',
       'marital_st', 'Lang_inside', 'Lang_outside', 'Education', 'lw_work',
       'lw_business', 'help_on_household', 'job_or_business', 'nature_of_work',
       'target'],
      dtype='object')

#### Data Visualisation

##### Visualize

In [6]:
sns.countplot(train_df["target"])

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [7]:
#checking null values
train_df.isna().sum()

ID                   0
gender               0
age                  0
race                 0
dwelling             0
dwelling_type        0
province_code        0
metro_code           0
psu                  0
nationality          0
RTH                  0
marital_st           0
Lang_inside          0
Lang_outside         0
Education            0
lw_work              0
lw_business          0
help_on_household    0
job_or_business      0
nature_of_work       0
target               0
dtype: int64

In [8]:
#Checking the correlation between the each feature and the target
train_df[train_df.columns[1:]].corr()['target'][:]

age      -0.085340
psu      -0.046302
target    1.000000
Name: target, dtype: float64

In [9]:
print(train_df['target'].value_counts())

0    7131
1     605
Name: target, dtype: int64


In [10]:
sns.countplot(train_df["job_or_business"])

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


- Try to create more visualisation from the other columns, what is the level of education of individual in the dataset.
- What about coorelation of other columns to the target.

##### Handling Categorical columns
- There are various way of encoding categorical columns. In the case below we're using label encoding 
<https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html>.
- Find out other ways of encoding categorical columns, like one hot encoding - which one works better on this case.

In [11]:
#Dropping unnecessary cols like gender
train_df = train_df.drop(['gender', 'ID', 'dwelling_type','nationality', 'help_on_household', 'lw_business','job_or_business',
 'Lang_inside', 'dwelling'], axis=1)

categorical_features = ['race', 'province_code', 'metro_code', 'RTH', 'marital_st', 'Lang_outside', 'Education','lw_work', 'nature_of_work']

In [12]:
train_df.shape

(7736, 12)

In [13]:
#Dealing with Categorical values using one hot encoding
train_df = pd.get_dummies(train_df, columns = categorical_features)

In [14]:
train_df

age       psu  ...  nature_of_work_Temporary  nature_of_work_Unspecified
0      20  66710095  ...                         0                           0
1      42  98510279  ...                         0                           0
2      86  29210321  ...                         0                           0
3      20  59913860  ...                         0                           0
4      26  79813828  ...                         0                           0
...   ...       ...  ...                       ...                         ...
7731   55  76011291  ...                         0                           0
7732   22  16910060  ...                         0                           0
7733   38  16410103  ...                         0                           1
7734   52  86810088  ...                         0                           0
7735   42  67410185  ...                         0                           0

[7736 rows x 110 columns]

##### Spliting the data
- It's always advisable to set aside some data for validating how your model performs. They are various methods of cross validation. In this case we use train-test split module from sklearn.

In [15]:
# spliting the data, 70% train set and 30% val set
train_set, val_set = train_test_split(train_df,
                               test_size=0.30, random_state=2)

In [16]:
train_set.shape

(5415, 110)

In [17]:
val_set.shape

(2321, 110)

In [18]:
X_train = train_set.drop(["target"], axis=1) # feature columns
y_train = train_set["target"] # the label/target column
X_val = val_set.drop(["target"], axis=1) 
y_val = val_set["target"] 

In [19]:
X_train

age       psu  ...  nature_of_work_Temporary  nature_of_work_Unspecified
2871   27  50310271  ...                         1                           0
1842   47  79810039  ...                         0                           0
6162   27  36210064  ...                         0                           0
7408   74  79712218  ...                         0                           0
4158   69  17810073  ...                         0                           0
...   ...       ...  ...                       ...                         ...
3606   49  79814311  ...                         0                           0
5704   36  79813255  ...                         1                           0
6637   53  79811411  ...                         0                           0
2575   42  19915916  ...                         0                           0
7336   39  26810054  ...                         0                           0

[5415 rows x 109 columns]

##### Model
- Using XGBoostRegressor

In [20]:
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=500, learning_rate=0.05, n_jobs =4, random_state =2)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_val, y_val)], 
             verbose=False)

[18:09:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=2,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [21]:
# export to 
import sklearn.externals
import joblib

joblib.dump(my_model, 'model.joblib')

['model.joblib']

###### 1. validation set
- let's see how our classifier performs on the validation dataset

In [22]:
pred =my_model.predict(X_val)
#print('score : ' , accuracy_score(y_val,pred))


###### 2. Error metric
 - f1 score is the error metric for this competition.
 - **f1 score** -  

In [23]:
#f1_score(y_val, pred)

#### Submission

- generating a sumission file 

###### 1. load the test data

- Do the same preprocessing performed on the training data

In [24]:
test_df = pd.read_csv("Test.csv")

test_df = test_df.drop(['gender', 'dwelling_type','nationality', 'help_on_household', 'lw_business','job_or_business',
 'Lang_inside', 'dwelling'], axis=1)

# Apply one-hot encoder to each column with categorical data
test_df = pd.get_dummies(test_df, columns = categorical_features)


In [25]:
### predict on the test data
test_ = test_df.drop(["ID"], axis=1)
test_predictions = my_model.predict(test_)
submission_df = pd.DataFrame()
submission_df["ID"] = test_df["ID"]
submission_df["target"] = test_predictions
# file
submission_df.to_csv("submission.csv", index=False)

In [26]:
submission = pd.read_csv("submission.csv")
submission

ID    target
0     ID_00241643  0.118677
1     ID_00312043  0.035491
2     ID_00345323  0.089999
3     ID_00369643  0.088227
4     ID_00381163  0.080097
...           ...       ...
3311  ID_99812843  0.100178
3312  ID_99815403  0.080292
3313  ID_99824363  0.122222
3314  ID_99832043  0.114620
3315  ID_99935723  0.137634

[3316 rows x 2 columns]